# 欠損値補完モデルで, testデータの欠損値を保管したデータにて学習


In [1]:
# セル 1: ライブラリのインポートとデータの読み込み・確認

# --- 1. 必要なライブラリのインポート ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_score, StratifiedGroupKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# --- 2. データの読み込み ---
# 指定されたパスから最終補完データを読み込む
final_data_path = '../data/all_data_imputed_step7_Spending.csv' # パスは環境に応じて調整してください
print(f"データを読み込んでいます: {final_data_path}")
all_data = pd.read_csv(final_data_path)
print("データ読み込み完了!")

# --- 3. データの基本情報確認 ---
print("\n--- データセットの基本情報 ---")
print(f"データ形状 (行, 列): {all_data.shape}")
print("\n列名:")
print(all_data.columns.tolist())
print("\nデータ型:")
print(all_data.dtypes)
print("\n先頭5行:")
print(all_data.head())
print("\n欠損値の確認:")
missing_values = all_data.isnull().sum()
print(missing_values[missing_values > 0])

# --- 4. 学習データとテストデータの分離 (Transported の有無で判断) ---
# Transported が NaN でない行が学習データ
train_df = all_data[all_data['Transported'].notna()].copy()
# Transported が NaN の行がテストデータ
test_df = all_data[all_data['Transported'].isna()].copy()

# 目的変数の型を bool に変換 (学習データのみ)
train_df['Transported'] = train_df['Transported'].astype(bool)

# --- 5. 整合性チェック: test.csv の PassengerId と all_data 内のテストデータ PassengerId が一致するか ---
print("\n--- 整合性チェック ---")
original_test_path = '../data/test.csv' # 元の test.csv のパス
original_test_df = pd.read_csv(original_test_path)
print(f"元の test.csv の行数: {len(original_test_df)}")
print(f"all_data から分離した test_df の行数: {len(test_df)}")

# PassengerId の集合を比較
set_original_test_ids = set(original_test_df['PassengerId'])
set_separated_test_ids = set(test_df['PassengerId'])

if set_original_test_ids == set_separated_test_ids:
    print("✓ 整合性チェック OK: test.csv の PassengerId と分離された test_df の PassengerId は一致しています。")
elif set_separated_test_ids.issuperset(set_original_test_ids):
    diff = set_separated_test_ids - set_original_test_ids
    print(f"! 注意: 分離された test_df に余分な PassengerId があります: {sorted(list(diff))[:5]}...") # 最初の5つだけ表示
else:
    diff = set_original_test_ids - set_separated_test_ids
    print(f"✗ 整合性チェック 失敗: test.csv に含まれるが分離された test_df にない PassengerId があります: {sorted(list(diff))[:5]}...") # 最初の5つだけ表示


# --- 6. 分割後のデータ確認 ---
print("\n--- 学習データとテストデータの分割 ---")
print(f"学習データ形状: {train_df.shape}")
print(f"テストデータ形状: {test_df.shape}")
print(f"学習データの目的変数 'Transported' の分布:")
print(train_df['Transported'].value_counts(dropna=False))


# 次のステップに進む準備ができました。
print("\n--- セル1完了 ---")
print("次のステップ: 特徴量エンジニアリングと前処理")

データを読み込んでいます: ../data/all_data_imputed_step7_Spending.csv
データ読み込み完了!

--- データセットの基本情報 ---
データ形状 (行, 列): (12970, 16)

列名:
['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported', 'Deck', 'Side']

データ型:
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported      object
Deck             object
Side             object
dtype: object

先頭5行:
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3 

/home/gohan/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# セル 2: 特徴量エンジニアリング (Feature Engineering)

# --- 1. 基本特徴量の選択 ---
feature_columns_base = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
    'Deck', 'Side',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'
]

# 学習データとテストデータから基本特徴量を抽出
X_train_base = train_df[feature_columns_base].copy()
X_test_base = test_df[feature_columns_base].copy()
y_train = train_df['Transported'].copy() # 目的変数

print("--- 基本特徴量の抽出 ---")
print(f"学習特徴量形状: {X_train_base.shape}")
print(f"テスト特徴量形状: {X_test_base.shape}")
print(f"目的変数形状: {y_train.shape}")

# --- 2. 派生特徴量の作成 ---
# --- 2.1. Total Spending ---
spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X_train_base['TotalSpending'] = X_train_base[spending_columns].sum(axis=1)
X_test_base['TotalSpending'] = X_test_base[spending_columns].sum(axis=1)

# --- 2.2. Log Total Spending (0を除外) ---
X_train_base['LogTotalSpending'] = np.log1p(X_train_base['TotalSpending']) # log1p は 0 を扱える
X_test_base['LogTotalSpending'] = np.log1p(X_test_base['TotalSpending'])

# --- 2.3. GroupID (PassengerId から抽出) ---
X_train_base['GroupID'] = train_df['PassengerId'].str.split('_').str[0]
X_test_base['GroupID'] = test_df['PassengerId'].str.split('_').str[0]

# --- 2.4. GroupSize (同じ GroupID の人数) ---
# GroupID の出現回数をカウント
group_counts_train = X_train_base['GroupID'].value_counts().to_dict()
group_counts_test = X_test_base['GroupID'].value_counts().to_dict()

# 訓練データとテストデータの GroupID カウントをマージ (GroupID が完全に別ならこれでOK)
# より正確には、all_data でカウントするのがベストだが、簡略化
X_train_base['GroupSize'] = X_train_base['GroupID'].map(group_counts_train)
X_test_base['GroupSize'] = X_test_base['GroupID'].map(group_counts_test)

# --- 2.5. AgeBand (年齢層ビン分け) ---
# ビンの境界を定義 (例: 0-12, 13-17, 18-25, 26-30, 31-50, 51+)
bins = [0, 12, 17, 25, 30, 50, 100]
labels = ['Child', 'Teen', 'Young_Adult', 'Adult', 'Mid_Age', 'Senior']
X_train_base['AgeBand'] = pd.cut(X_train_base['Age'], bins=bins, labels=labels, right=False)
X_test_base['AgeBand'] = pd.cut(X_test_base['Age'], bins=bins, labels=labels, right=False)

# --- 2.6. PlanetRoute (HomePlanet x Destination) ---
X_train_base['PlanetRoute'] = X_train_base['HomePlanet'].astype(str) + "_to_" + X_train_base['Destination'].astype(str)
X_test_base['PlanetRoute'] = X_test_base['HomePlanet'].astype(str) + "_to_" + X_test_base['Destination'].astype(str)

# --- 2.7. SpendingPerAge (例: TotalSpending / Age, Age>0) ---
# np.where を使って、Ageが0の場合は NaN または 0 にするなどの処理を入れる
X_train_base['SpendingPerAge'] = np.where(X_train_base['Age'] > 0, X_train_base['TotalSpending'] / X_train_base['Age'], 0)
X_test_base['SpendingPerAge'] = np.where(X_test_base['Age'] > 0, X_test_base['TotalSpending'] / X_test_base['Age'], 0)

# --- 3. エンコーディングの準備 ---
# GroupID は CV 分割にのみ使用し、モデルには入力しない
features_for_encoding = X_train_base.columns.drop('GroupID').tolist()
print(f"\n--- エンコーディング対象の特徴量 ---")
print(features_for_encoding)

# 学習データとテストデータを結合して、一貫したエンコーディングを行う
combined_features_raw = pd.concat([X_train_base[features_for_encoding], X_test_base[features_for_encoding]], axis=0, ignore_index=True)
print(f"\n結合後特徴量形状: {combined_features_raw.shape}")

# --- 4. One-Hot Encoding ---
# カテゴリ変数を指定 (数値型以外 or 明示的に指定)
categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'Side', 'AgeBand', 'PlanetRoute']
# VIP は bool 型なので、必要に応じて変換 (ここではそのまま bool として扱う)

# One-Hot Encoding を適用 (dummy_na=True で NaN もカテゴリとして扱うが、今回のデータでは欠損なし)
combined_features_encoded = pd.get_dummies(combined_features_raw, columns=categorical_features, dummy_na=False, drop_first=False)

# --- 5. エンコード後のデータを学習用とテスト用に再分割 ---
X_train_processed = combined_features_encoded.iloc[:len(X_train_base)].reset_index(drop=True)
X_test_processed = combined_features_encoded.iloc[len(X_train_base):].reset_index(drop=True)

# 目的変数 y_train もリセットインデックス
y_train = y_train.reset_index(drop=True)

print(f"\n--- エンコード後のデータ形状 ---")
print(f"学習特徴量 (X_train_processed) 形状: {X_train_processed.shape}")
print(f"テスト特徴量 (X_test_processed) 形状: {X_test_processed.shape}")
print(f"目的変数 (y_train) 形状: {y_train.shape}")

# --- 6. 使用する特徴量カラムの最終確認 ---
final_feature_columns = X_train_processed.columns.tolist()
print(f"\n最終特徴量数: {len(final_feature_columns)}")
# print("最終特徴量カラム名:")
# print(final_feature_columns) # 長いので必要時のみコメントアウトを外す

print("\n--- セル2完了 ---")
print("次のステップ: モデルの選定・学習・評価")


--- 基本特徴量の抽出 ---
学習特徴量形状: (8693, 12)
テスト特徴量形状: (4277, 12)
目的変数形状: (8693,)

--- エンコーディング対象の特徴量 ---
['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'Deck', 'Side', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpending', 'LogTotalSpending', 'GroupSize', 'AgeBand', 'PlanetRoute', 'SpendingPerAge']

結合後特徴量形状: (12970, 18)

--- エンコード後のデータ形状 ---
学習特徴量 (X_train_processed) 形状: (8693, 46)
テスト特徴量 (X_test_processed) 形状: (4277, 46)
目的変数 (y_train) 形状: (8693,)

最終特徴量数: 46

--- セル2完了 ---
次のステップ: モデルの選定・学習・評価


In [4]:
# セル 3: モデルの選定・学習・評価 (LightGBM + Optuna + StratifiedGroupKFold)

# --- 1. モデルとCVの設定 ---
import lightgbm as lgb
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

# GroupID を整数に変換 (StratifiedGroupKFold 用)
# X_train_processed に GroupID がないので、元の X_train_base から取得
groups = X_train_base['GroupID'].reset_index(drop=True) # インデックスをリセットしてから取得
groups_int = pd.to_numeric(groups, errors='coerce').fillna(0).astype(int)

# StratifiedGroupKFold の設定
# 知識ベースによると CV vs LB ギャップがあるので、SGKFold を使用
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# --- 2. Optunaによるハイパーパラメーターチューニング ---
def objective(trial):
    """
    Optuna の目的関数。LightGBM のパラメータを提案し、CVスコアを返す。
    """
    param = {
        "objective": "binary",
        "metric": "binary_logloss", # Optuna の maximize/minimize は accuracy には向いていないため、logloss を最小化する
        "verbosity": -1,
        "boosting_type": "gbdt",
        # "device_type": "gpu", # GPU を使用する場合はコメントアウトを外す (環境依存)
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True), # logスケールで探索
    }

    # CVスコアを格納するリスト
    cv_scores = []

    # StratifiedGroupKFold で分割
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_processed, y_train, groups=groups_int)):
        # データ分割
        X_train_fold = X_train_processed.iloc[train_idx]
        y_train_fold = y_train.iloc[train_idx]
        X_val_fold = X_train_processed.iloc[val_idx]
        y_val_fold = y_train.iloc[val_idx]

        # LightGBM Dataset の作成
        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)

        # モデル学習
        model = lgb.train(
            param,
            train_data,
            valid_sets=[val_data],
            valid_names=['valid'],
            num_boost_round=1000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=False),
                lgb.log_evaluation(period=0) # 学習ログを非表示
            ]
        )

        # 予測 (確率)
        y_pred_proba = model.predict(X_val_fold, num_iteration=model.best_iteration)
        # 確率をラベルに変換 (0.5 以上を True)
        y_pred = (y_pred_proba >= 0.5).astype(int)

        # Accuracy を計算
        acc = accuracy_score(y_val_fold, y_pred)
        cv_scores.append(acc)

    # 平均CVスコアを返す (Optuna はデフォルトで minimize するので、-1をかける)
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score # Accuracy を直接最大化するため、マイナスはつけない

# Optuna のスタディを作成
study = optuna.create_study(direction='maximize') # Accuracy を最大化
print("Optuna チューニングを開始します...")
# 試行回数を制限 (例: 50回) して実行時間をコントロール
study.optimize(objective, n_trials=50) # 実際には100~200程度が望ましいが、ここでは50回で様子見

print("\n--- Optuna チューニング結果 ---")
print("最適なパラメータ:", study.best_params)
print("最適なCVスコア (Accuracy):", study.best_value)

# --- 3. 最適パラメータでモデルを学習 ---
best_params = study.best_params
best_params["objective"] = "binary"
best_params["metric"] = "binary_logloss"
best_params["verbosity"] = -1
# best_params["device_type"] = "gpu" # GPU 使用時

# 全データで学習 (CVの設定は不要)
final_train_data = lgb.Dataset(X_train_processed, label=y_train)
# --- 修正: callbacks から early_stopping を削除 ---
final_model = lgb.train(
    best_params,
    final_train_data,
    num_boost_round=1000, # 固定のiteration数、または study.best_trial から取得した平均 best_iteration を使う
    callbacks=[
        # lgb.early_stopping(stopping_rounds=100, verbose=True), # 削除
        lgb.log_evaluation(period=100)
    ]
)

print("\n--- モデル学習完了 ---")

# --- 4. テストデータへの予測 ---
# 確率で予測
y_test_pred_proba = final_model.predict(X_test_processed, num_iteration=final_model.best_iteration)
# 0.5 を閾値としてラベルに変換
y_test_pred = (y_test_pred_proba >= 0.5).astype(bool)

print("\n--- テストデータ予測完了 ---")
print(f"予測された Transported の分布:")
print(pd.Series(y_test_pred).value_counts())

# --- 5. 提出ファイルの作成 ---
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_test_pred
})

# 提出ファイルを保存
submission_path = '../outputs/submissions/ver5/soya_model5.csv'
submission.to_csv(submission_path, index=False)
print(f"\n提出ファイルを保存しました: {submission_path}")

print("\n--- セル3完了 ---")
print("次のステップ: 結果の確認と提出")



[I 2025-07-25 23:13:15,427] A new study created in memory with name: no-name-fe666aaa-d944-4c96-b830-7fb07ee41238


Optuna チューニングを開始します...


[I 2025-07-25 23:13:18,025] Trial 0 finished with value: 0.802193521502045 and parameters: {'lambda_l1': 0.007598412497576287, 'lambda_l2': 0.04462337491026621, 'num_leaves': 143, 'feature_fraction': 0.9394483708078196, 'bagging_fraction': 0.46780270170600136, 'bagging_freq': 1, 'min_child_samples': 39, 'min_gain_to_split': 4.595296993001769, 'learning_rate': 0.052536767526617095}. Best is trial 0 with value: 0.802193521502045.
[I 2025-07-25 23:13:23,383] Trial 1 finished with value: 0.8042699600916954 and parameters: {'lambda_l1': 5.653250933369594e-06, 'lambda_l2': 3.11227845169353e-06, 'num_leaves': 255, 'feature_fraction': 0.6316752717927361, 'bagging_fraction': 0.8809764022479475, 'bagging_freq': 1, 'min_child_samples': 59, 'min_gain_to_split': 2.568817254841656, 'learning_rate': 0.010459067964705627}. Best is trial 1 with value: 0.8042699600916954.
[I 2025-07-25 23:13:26,011] Trial 2 finished with value: 0.8008878268279478 and parameters: {'lambda_l1': 5.638342677328735e-08, 'lam


--- Optuna チューニング結果 ---
最適なパラメータ: {'lambda_l1': 0.02401760408646057, 'lambda_l2': 0.008131454157467705, 'num_leaves': 92, 'feature_fraction': 0.5571261663518792, 'bagging_fraction': 0.5735762581660586, 'bagging_freq': 1, 'min_child_samples': 12, 'min_gain_to_split': 0.9020076179879019, 'learning_rate': 0.008074542324563254}
最適なCVスコア (Accuracy): 0.8055313958940129

--- モデル学習完了 ---

--- テストデータ予測完了 ---
予測された Transported の分布:
True     2215
False    2062
Name: count, dtype: int64


OSError: Cannot save file into a non-existent directory: '../outputs/submissions/ver5'

In [7]:
# セル 4: XGBoost モデルの構築と予測

# --- 1. 必要なライブラリのインポート ---
import xgboost as xgb
import optuna
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd # pd.Series で使用

# --- 2. XGBoost 用の Optuna 目的関数の定義 ---
def objective_xgb(trial):
    """
    Optuna の目的関数 (XGBoost 用)。パラメータを提案し、StratifiedGroupKFold での CV スコア (Accuracy) を返す。
    """
    # XGBoost パラメータの提案
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss', # Optuna 内部で最小化する指標
        'booster': 'gbtree',
        # 'tree_method': 'gpu_hist', # GPU を使用する場合はコメントを外す
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True), # L2 正則化
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True), # L1 正則化
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0), # 列のサブサンプリング
        'subsample': trial.suggest_float('subsample', 0.4, 1.0), # 訓練データの行のサブサンプリング
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # 学習率
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # 木の数 (iteration数)
        'max_depth': trial.suggest_int('max_depth', 3, 12), # 木の深さ
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300), # 子ノードに必要な重みの最小値
    }
    
    # StratifiedGroupKFold の設定 (LightGBM と同じ)
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    
    # CVスコアを格納するリスト
    cv_scores = []
    
    # StratifiedGroupKFold で分割
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_processed, y_train, groups=groups_int)):
        # データ分割
        X_train_fold = X_train_processed.iloc[train_idx]
        y_train_fold = y_train.iloc[train_idx]
        X_val_fold = X_train_processed.iloc[val_idx]
        y_val_fold = y_train.iloc[val_idx]

        # XGBoost モデルのインスタンス作成
        model = xgb.XGBClassifier(**param, random_state=42, verbosity=0)
        
        # --- 修正箇所 ---
        # early_stopping を使わず、n_estimators をチューニングに任せる
        # eval_set は指定可能だが、early_stopping_rounds と callbacks は使わない
        # --- 修正ここまで ---

        # モデル学習
        model.fit(
            X_train_fold, y_train_fold,
            # eval_set=[(X_val_fold, y_val_fold)], # 評価セットの指定は可能
            verbose=False
            # early_stopping_rounds=100, # 削除
            # callbacks=[...] # 削除
        )

        # 予測 (確率)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1] # True の確率を取得
        # 確率をラベルに変換 (0.5 以上を True)
        y_pred = (y_pred_proba >= 0.5).astype(int)

        # Accuracy を計算
        acc = accuracy_score(y_val_fold, y_pred)
        cv_scores.append(acc)

    # 平均CVスコアを返す (Accuracy を最大化)
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score

# --- 3. Optuna による XGBoost のハイパーパラメーターチューニング ---
study_xgb = optuna.create_study(direction='maximize') # Accuracy を最大化
print("Optuna チューニングを開始します (XGBoost)...")
# 試行回数を制限 (例: 30回) して実行時間をコントロール
study_xgb.optimize(objective_xgb, n_trials=30) # 実際には50~100程度が望ましい

print("\n--- Optuna チューニング結果 (XGBoost) ---")
print("最適なパラメータ:", study_xgb.best_params)
print("最適なCVスコア (Accuracy):", study_xgb.best_value)

# --- 4. 最適パラメータで XGBoost モデルを学習 ---
best_params_xgb = study_xgb.best_params
# best_params_xgb['tree_method'] = 'gpu_hist' # GPU 使用時

# 全データで学習
final_model_xgb = xgb.XGBClassifier(**best_params_xgb, random_state=42, verbosity=1)

print("\nXGBoost モデルを全データで学習中...")
final_model_xgb.fit(X_train_processed, y_train)
print("--- XGBoost モデル学習完了 ---")

# --- 5. テストデータへの予測 ---
# 確率で予測
y_test_pred_proba_xgb = final_model_xgb.predict_proba(X_test_processed)[:, 1] # True の確率
# 0.5 を閾値としてラベルに変換
y_test_pred_xgb = (y_test_pred_proba_xgb >= 0.5).astype(bool)

print("\n--- XGBoost テストデータ予測完了 ---")
print(f"予測された Transported の分布:")
print(pd.Series(y_test_pred_xgb).value_counts())

# --- 6. XGBoost の予測結果を保存 (アンサンブル用) ---
# 次のアンサンブルステップで使うために、確率も保存しておく
np.save('../outputs/oof_xgb_proba.npy', y_test_pred_proba_xgb) # 今回は OOF ではないが、便宜上
print("\nXGBoost の予測確率を保存しました: ../outputs/oof_xgb_proba.npy")

print("\n--- セル4完了 ---")
print("次のステップ: 他のモデル (例: CatBoost) や アンサンブル")


[I 2025-07-25 23:32:56,482] A new study created in memory with name: no-name-81742b2d-b865-4dc8-aa25-96370e9241cf


Optuna チューニングを開始します (XGBoost)...


[I 2025-07-25 23:33:00,314] Trial 0 finished with value: 0.7877112755031714 and parameters: {'lambda': 0.1592104435040895, 'alpha': 1.2299473567219701e-06, 'colsample_bytree': 0.4152656145740945, 'subsample': 0.9731882244224208, 'learning_rate': 0.008916214523288413, 'n_estimators': 538, 'max_depth': 7, 'min_child_weight': 147}. Best is trial 0 with value: 0.7877112755031714.
[I 2025-07-25 23:33:03,236] Trial 1 finished with value: 0.7952116363103086 and parameters: {'lambda': 1.4077453553473529e-05, 'alpha': 3.9967678426707544e-07, 'colsample_bytree': 0.6153785590682018, 'subsample': 0.9796432590908821, 'learning_rate': 0.1960994366731847, 'n_estimators': 560, 'max_depth': 6, 'min_child_weight': 18}. Best is trial 1 with value: 0.7952116363103086.
[I 2025-07-25 23:33:05,970] Trial 2 finished with value: 0.7938282633582667 and parameters: {'lambda': 0.00012559841033867112, 'alpha': 4.056726595749755e-08, 'colsample_bytree': 0.5745175962601097, 'subsample': 0.6661925898431725, 'learning


--- Optuna チューニング結果 (XGBoost) ---
最適なパラメータ: {'lambda': 2.1951709666388663e-08, 'alpha': 8.547169971217096e-08, 'colsample_bytree': 0.5521475730416037, 'subsample': 0.8615268396596035, 'learning_rate': 0.11366011585971071, 'n_estimators': 272, 'max_depth': 4, 'min_child_weight': 23}
最適なCVスコア (Accuracy): 0.802688759745469

XGBoost モデルを全データで学習中...
--- XGBoost モデル学習完了 ---

--- XGBoost テストデータ予測完了 ---
予測された Transported の分布:
True     2224
False    2053
Name: count, dtype: int64

XGBoost の予測確率を保存しました: ../outputs/oof_xgb_proba.npy

--- セル4完了 ---
次のステップ: 他のモデル (例: CatBoost) や アンサンブル


In [9]:
# セル 5: CatBoost モデルの構築と予測

# --- 1. 必要なライブラリのインポート ---
import catboost as cb
import optuna
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd # pd.Series で使用

# --- 2. CatBoost 用の Optuna 目的関数の定義 ---
def objective_cat(trial):
    """
    Optuna の目的関数 (CatBoost 用)。パラメータを提案し、StratifiedGroupKFold での CV スコア (Accuracy) を返す。
    """
    # CatBoost パラメータの基本部分を提案
    param = {
        'objective': 'Logloss', # 二値分類
        'eval_metric': 'Accuracy', # 評価指標
        # 'task_type': "GPU", # GPU を使用する場合はコメントを外す
        'devices': '0', # GPU デバイス番号 (GPU使用時)
        'iterations': trial.suggest_int('iterations', 100, 1000), # 木の数
        'depth': trial.suggest_int('depth', 3, 10), # 木の深さ
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # 学習率
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # L2 正則化係数
        'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True), # ランダム化の強さ
        'od_type': 'Iter', # overfitting detector type
        'od_wait': 100, # overfitting detector が待機するラウンド数
        'verbose': False, # 学習ログを非表示
        'random_seed': 42,
    }
    
    # bootstrap_type を選択
    bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS'])
    param['bootstrap_type'] = bootstrap_type
    
    # bootstrap_type に応じて追加パラメータを設定 (完全に分離)
    if bootstrap_type == 'Bayesian':
        param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0.0, 10.0)
    elif bootstrap_type == 'Bernoulli':
        param['subsample'] = trial.suggest_float('subsample', 0.1, 1.0)
    # MVS の場合は追加パラメータなし

    # StratifiedGroupKFold の設定 (LightGBM/XGBoost と同じ)
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    
    # CVスコアを格納するリスト
    cv_scores = []
    
    # StratifiedGroupKFold で分割
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_processed, y_train, groups=groups_int)):
        # データ分割
        X_train_fold = X_train_processed.iloc[train_idx]
        y_train_fold = y_train.iloc[train_idx]
        X_val_fold = X_train_processed.iloc[val_idx]
        y_val_fold = y_train.iloc[val_idx]

        # CatBoost モデルのインスタンス作成
        model = cb.CatBoostClassifier(**param)
        
        # モデル学習 (eval_set を指定して overfitting detector を有効化)
        model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            early_stopping_rounds=100, # od_wait と組み合わせて使用
            verbose=False
        )

        # 予測 (確率)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1] # True の確率を取得
        # 確率をラベルに変換 (0.5 以上を True)
        y_pred = (y_pred_proba >= 0.5).astype(int)

        # Accuracy を計算
        acc = accuracy_score(y_val_fold, y_pred)
        cv_scores.append(acc)

    # 平均CVスコアを返す (Accuracy を最大化)
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score

# --- 3. Optuna による CatBoost のハイパーパラメーターチューニング ---
study_cat = optuna.create_study(direction='maximize') # Accuracy を最大化
print("Optuna チューニングを開始します (CatBoost)...")
# 試行回数を制限 (例: 20~30回) して実行時間をコントロール (CatBoost は少し重いので)
study_cat.optimize(objective_cat, n_trials=25) # 実際には50~100程度が望ましいが、ここでは25回

print("\n--- Optuna チューニング結果 (CatBoost) ---")
print("最適なパラメータ:", study_cat.best_params)
print("最適なCVスコア (Accuracy):", study_cat.best_value)

# --- 4. 最適パラメータで CatBoost モデルを学習 ---
best_params_cat = study_cat.best_params
# best_params_cat['task_type'] = "GPU" # GPU 使用時
# best_params_cat['devices'] = '0' # GPU 使用時

# 全データで学習 (overfitting detector は使用しない、またはホールドアウトで設定)
# od_type と od_wait は学習時にも指定する必要がある場合があるが、最終学習では省略可能
final_model_cat = cb.CatBoostClassifier(
    **best_params_cat,
    verbose=False,
    random_seed=42
)

print("\nCatBoost モデルを全データで学習中...")
final_model_cat.fit(X_train_processed, y_train)
print("--- CatBoost モデル学習完了 ---")

# --- 5. テストデータへの予測 ---
# 確率で予測
y_test_pred_proba_cat = final_model_cat.predict_proba(X_test_processed)[:, 1] # True の確率
# 0.5 を閾値としてラベルに変換
y_test_pred_cat = (y_test_pred_proba_cat >= 0.5).astype(bool)

print("\n--- CatBoost テストデータ予測完了 ---")
print(f"予測された Transported の分布:")
print(pd.Series(y_test_pred_cat).value_counts())

# --- 6. CatBoost の予測結果を保存 (アンサンブル用) ---
# 次のアンサンブルステップで使うために、確率も保存しておく
np.save('../outputs/oof_cat_proba.npy', y_test_pred_proba_cat) # 今回は OOF ではないが、便宜上
print("\nCatBoost の予測確率を保存しました: ../outputs/oof_cat_proba.npy")

print("\n--- セル5完了 ---")
print("次のステップ: 他のモデル (例: HistGradientBoostingClassifier) や アンサンブル")


[I 2025-07-25 23:37:29,535] A new study created in memory with name: no-name-aa99ec51-5cf9-4803-9452-f41675b18cf1


Optuna チューニングを開始します (CatBoost)...


[I 2025-07-25 23:37:32,312] Trial 0 finished with value: 0.8089854374228915 and parameters: {'iterations': 613, 'depth': 4, 'learning_rate': 0.22496430236300835, 'l2_leaf_reg': 0.036720348045743006, 'random_strength': 0.1483146157343797, 'bootstrap_type': 'Bernoulli', 'subsample': 0.5046107332572124}. Best is trial 0 with value: 0.8089854374228915.
[I 2025-07-25 23:37:41,047] Trial 1 finished with value: 0.8051665642188954 and parameters: {'iterations': 242, 'depth': 9, 'learning_rate': 0.10855081672517695, 'l2_leaf_reg': 0.006212426086251139, 'random_strength': 5.906121210869269, 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.8089854374228915.
[I 2025-07-25 23:37:46,289] Trial 2 finished with value: 0.7501306639449714 and parameters: {'iterations': 618, 'depth': 7, 'learning_rate': 0.0010143217479284365, 'l2_leaf_reg': 1.979819661537661e-06, 'random_strength': 6.121966623881717, 'bootstrap_type': 'Bernoulli', 'subsample': 0.5163786149933159}. Best is trial 0 with value: 0.808


--- Optuna チューニング結果 (CatBoost) ---
最適なパラメータ: {'iterations': 228, 'depth': 5, 'learning_rate': 0.18703658353095073, 'l2_leaf_reg': 0.0015813751550870004, 'random_strength': 0.0027365162272571797, 'bootstrap_type': 'Bernoulli', 'subsample': 0.8645485289904373}
最適なCVスコア (Accuracy): 0.8133362587810495

CatBoost モデルを全データで学習中...
--- CatBoost モデル学習完了 ---

--- CatBoost テストデータ予測完了 ---
予測された Transported の分布:
True     2197
False    2080
Name: count, dtype: int64

CatBoost の予測確率を保存しました: ../outputs/oof_cat_proba.npy

--- セル5完了 ---
次のステップ: 他のモデル (例: HistGradientBoostingClassifier) や アンサンブル


In [10]:
# セル 6: HistGradientBoostingClassifier モデルの構築と予測

# --- 1. 必要なライブラリのインポート ---
from sklearn.ensemble import HistGradientBoostingClassifier
import optuna
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import numpy as np

# --- 2. HistGradientBoostingClassifier 用の Optuna 目的関数の定義 ---
def objective_hgb(trial):
    """
    Optuna の目的関数 (HistGradientBoostingClassifier 用)。
    パラメータを提案し、StratifiedGroupKFold での CV スコア (Accuracy) を返す。
    """
    # HistGradientBoostingClassifier パラメータの提案
    param = {
        'loss': 'log_loss', # 二値分類
        # 'device': 'cuda', # GPU を使用する場合はコメントを外す (sklearn >= 1.5 が必要)
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # 学習率
        'max_iter': trial.suggest_int('max_iter', 100, 1000), # 最大イテレーション数
        'max_depth': trial.suggest_int('max_depth', 3, 12), # 木の最大深さ
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-8, 10.0, log=True), # L2 正則化
        'early_stopping': True, # 早期停止を有効化
        'validation_fraction': 0.1, # 検証用データの割合 (early_stopping 用)
        'n_iter_no_change': 50, # 早期停止の待ちラウンド数
        'tol': 1e-7, # 早期停止の許容誤差
        'random_state': 42,
        'verbose': 0, # 学習ログを非表示
    }
    
    # StratifiedGroupKFold の設定 (他のモデルと同じ)
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    
    # CVスコアを格納するリスト
    cv_scores = []
    
    # StratifiedGroupKFold で分割 (注意: HistGradientBoosting は内部でホールドアウトしない)
    # したがって、ここでは通常の StratifiedGroupKFold で評価する
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_processed, y_train, groups=groups_int)):
        # データ分割
        X_train_fold = X_train_processed.iloc[train_idx]
        y_train_fold = y_train.iloc[train_idx]
        X_val_fold = X_train_processed.iloc[val_idx]
        y_val_fold = y_train.iloc[val_idx]

        # HistGradientBoostingClassifier モデルのインスタンス作成
        model = HistGradientBoostingClassifier(**param)
        
        # モデル学習 (early_stopping は内部で validation_fraction に基づいて処理される)
        model.fit(X_train_fold, y_train_fold)

        # 予測 (確率)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1] # True の確率を取得
        # 確率をラベルに変換 (0.5 以上を True)
        y_pred = (y_pred_proba >= 0.5).astype(int)

        # Accuracy を計算
        acc = accuracy_score(y_val_fold, y_pred)
        cv_scores.append(acc)

    # 平均CVスコアを返す (Accuracy を最大化)
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score

# --- 3. Optuna による HistGradientBoostingClassifier のハイパーパラメーターチューニング ---
study_hgb = optuna.create_study(direction='maximize') # Accuracy を最大化
print("Optuna チューニングを開始します (HistGradientBoostingClassifier)...")
# 試行回数を制限 (例: 20~30回)
study_hgb.optimize(objective_hgb, n_trials=25) # 実際には50~100程度が望ましいが、ここでは25回

print("\n--- Optuna チューニング結果 (HistGradientBoostingClassifier) ---")
print("最適なパラメータ:", study_hgb.best_params)
print("最適なCVスコア (Accuracy):", study_hgb.best_value)

# --- 4. 最適パラメータで HistGradientBoostingClassifier モデルを学習 ---
best_params_hgb = study_hgb.best_params
# best_params_hgb['device'] = 'cuda' # GPU 使用時
best_params_hgb['early_stopping'] = False # 最終学習では早期停止を無効化し、全データを使う
best_params_hgb['verbose'] = 0
best_params_hgb['random_state'] = 42

# 全データで学習
final_model_hgb = HistGradientBoostingClassifier(**best_params_hgb)

print("\nHistGradientBoostingClassifier モデルを全データで学習中...")
final_model_hgb.fit(X_train_processed, y_train)
print("--- HistGradientBoostingClassifier モデル学習完了 ---")

# --- 5. テストデータへの予測 ---
# 確率で予測
y_test_pred_proba_hgb = final_model_hgb.predict_proba(X_test_processed)[:, 1] # True の確率
# 0.5 を閾値としてラベルに変換
y_test_pred_hgb = (y_test_pred_proba_hgb >= 0.5).astype(bool)

print("\n--- HistGradientBoostingClassifier テストデータ予測完了 ---")
print(f"予測された Transported の分布:")
print(pd.Series(y_test_pred_hgb).value_counts())

# --- 6. HistGradientBoostingClassifier の予測結果を保存 (アンサンブル用) ---
# 次のアンサンブルステップで使うために、確率も保存しておく
np.save('../outputs/oof_hgb_proba.npy', y_test_pred_proba_hgb) # 今回は OOF ではないが、便宜上
print("\nHistGradientBoostingClassifier の予測確率を保存しました: ../outputs/oof_hgb_proba.npy")

print("\n--- セル6完了 ---")
print("次のステップ: 他のモデル (例: RandomForestClassifier) や アンサンブルの準備")


[I 2025-07-25 23:42:35,399] A new study created in memory with name: no-name-6e743167-40fc-4020-89c3-628f6fe93d1b


Optuna チューニングを開始します (HistGradientBoostingClassifier)...


[I 2025-07-25 23:42:40,073] Trial 0 finished with value: 0.8052881825169355 and parameters: {'learning_rate': 0.011120093135150428, 'max_iter': 533, 'max_depth': 12, 'l2_regularization': 0.0022642721301332188}. Best is trial 0 with value: 0.8052881825169355.
[I 2025-07-25 23:42:47,396] Trial 1 finished with value: 0.797137527921844 and parameters: {'learning_rate': 0.0014647298840029487, 'max_iter': 722, 'max_depth': 8, 'l2_regularization': 0.007195799265813361}. Best is trial 0 with value: 0.8052881825169355.
[I 2025-07-25 23:42:51,415] Trial 2 finished with value: 0.7962083433682471 and parameters: {'learning_rate': 0.0028096192830954964, 'max_iter': 300, 'max_depth': 8, 'l2_regularization': 0.05161328841015196}. Best is trial 0 with value: 0.8052881825169355.
[I 2025-07-25 23:42:54,686] Trial 3 finished with value: 0.8053460406643289 and parameters: {'learning_rate': 0.02952584900641574, 'max_iter': 565, 'max_depth': 9, 'l2_regularization': 0.06895299621458444}. Best is trial 3 with


--- Optuna チューニング結果 (HistGradientBoostingClassifier) ---
最適なパラメータ: {'learning_rate': 0.03466690134931694, 'max_iter': 836, 'max_depth': 10, 'l2_regularization': 0.2163254731996442}
最適なCVスコア (Accuracy): 0.8054180486455309

HistGradientBoostingClassifier モデルを全データで学習中...
--- HistGradientBoostingClassifier モデル学習完了 ---

--- HistGradientBoostingClassifier テストデータ予測完了 ---
予測された Transported の分布:
True     2186
False    2091
Name: count, dtype: int64

HistGradientBoostingClassifier の予測確率を保存しました: ../outputs/oof_hgb_proba.npy

--- セル6完了 ---
次のステップ: 他のモデル (例: RandomForestClassifier) や アンサンブルの準備


In [11]:
# セル 7: RandomForestClassifier モデルの構築と予測

# --- 1. 必要なライブラリのインポート ---
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import numpy as np

# --- 2. RandomForestClassifier 用の Optuna 目的関数の定義 ---
def objective_rf(trial):
    """
    Optuna の目的関数 (RandomForestClassifier 用)。
    パラメータを提案し、StratifiedGroupKFold での CV スコア (Accuracy) を返す。
    """
    # RandomForestClassifier パラメータの提案
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # 決定木の数
        'max_depth': trial.suggest_int('max_depth', 3, 20), # 木の最大深さ
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100), # 内部ノードを分割するのに必要な最小サンプル数
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100), # 葉ノードに必要な最小サンプル数
        'max_features': trial.suggest_float('max_features', 0.1, 1.0), # 各分割で考慮する特徴量の割合
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]), # ブートストラップサンプリングを行うか
        # 'oob_score': True, # Out-of-Bag スコアを計算 (early stopping の代替になる可能性あり)
        'random_state': 42,
        'n_jobs': -1, # 並列処理
        'verbose': 0, # 学習ログを非表示
    }
    
    # StratifiedGroupKFold の設定 (他のモデルと同じ)
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    
    # CVスコアを格納するリスト
    cv_scores = []
    
    # StratifiedGroupKFold で分割
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_processed, y_train, groups=groups_int)):
        # データ分割
        X_train_fold = X_train_processed.iloc[train_idx]
        y_train_fold = y_train.iloc[train_idx]
        X_val_fold = X_train_processed.iloc[val_idx]
        y_val_fold = y_train.iloc[val_idx]

        # RandomForestClassifier モデルのインスタンス作成
        model = RandomForestClassifier(**param)
        
        # モデル学習
        model.fit(X_train_fold, y_train_fold)

        # 予測 (確率)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1] # True の確率を取得
        # 確率をラベルに変換 (0.5 以上を True)
        y_pred = (y_pred_proba >= 0.5).astype(int)

        # Accuracy を計算
        acc = accuracy_score(y_val_fold, y_pred)
        cv_scores.append(acc)

    # 平均CVスコアを返す (Accuracy を最大化)
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score

# --- 3. Optuna による RandomForestClassifier のハイパーパラメーターチューニング ---
study_rf = optuna.create_study(direction='maximize') # Accuracy を最大化
print("Optuna チューニングを開始します (RandomForestClassifier)...")
# 試行回数を制限 (例: 20~30回) RandomForest は比較的重いので
study_rf.optimize(objective_rf, n_trials=20) # 実際には50~100程度が望ましいが、ここでは20回

print("\n--- Optuna チューニング結果 (RandomForestClassifier) ---")
print("最適なパラメータ:", study_rf.best_params)
print("最適なCVスコア (Accuracy):", study_rf.best_value)

# --- 4. 最適パラメータで RandomForestClassifier モデルを学習 ---
best_params_rf = study_rf.best_params
best_params_rf['random_state'] = 42
best_params_rf['n_jobs'] = -1
best_params_rf['verbose'] = 0

# 全データで学習
final_model_rf = RandomForestClassifier(**best_params_rf)

print("\nRandomForestClassifier モデルを全データで学習中...")
final_model_rf.fit(X_train_processed, y_train)
print("--- RandomForestClassifier モデル学習完了 ---")

# --- 5. テストデータへの予測 ---
# 確率で予測
y_test_pred_proba_rf = final_model_rf.predict_proba(X_test_processed)[:, 1] # True の確率
# 0.5 を閾値としてラベルに変換
y_test_pred_rf = (y_test_pred_proba_rf >= 0.5).astype(bool)

print("\n--- RandomForestClassifier テストデータ予測完了 ---")
print(f"予測された Transported の分布:")
print(pd.Series(y_test_pred_rf).value_counts())

# --- 6. RandomForestClassifier の予測結果を保存 (アンサンブル用) ---
# 次のアンサンブルステップで使うために、確率も保存しておく
np.save('../outputs/oof_rf_proba.npy', y_test_pred_proba_rf) # 今回は OOF ではないが、便宜上
print("\nRandomForestClassifier の予測確率を保存しました: ../outputs/oof_rf_proba.npy")

print("\n--- セル7完了 ---")
print("次のステップ: アンサンブル (モデルの予測確率の平均化など)")


[I 2025-07-25 23:45:47,507] A new study created in memory with name: no-name-56610711-b64b-462a-ad5c-a605ccd2191f


Optuna チューニングを開始します (RandomForestClassifier)...


[I 2025-07-25 23:45:49,881] Trial 0 finished with value: 0.7970889236785873 and parameters: {'n_estimators': 214, 'max_depth': 9, 'min_samples_split': 75, 'min_samples_leaf': 54, 'max_features': 0.5186479351030339, 'bootstrap': False}. Best is trial 0 with value: 0.7970889236785873.
[I 2025-07-25 23:45:57,300] Trial 1 finished with value: 0.7958044862437375 and parameters: {'n_estimators': 783, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 42, 'max_features': 0.42191071870164465, 'bootstrap': True}. Best is trial 0 with value: 0.7970889236785873.
[I 2025-07-25 23:46:05,986] Trial 2 finished with value: 0.7883236253203936 and parameters: {'n_estimators': 888, 'max_depth': 18, 'min_samples_split': 53, 'min_samples_leaf': 59, 'max_features': 0.19326703912015103, 'bootstrap': True}. Best is trial 0 with value: 0.7970889236785873.
[I 2025-07-25 23:46:12,716] Trial 3 finished with value: 0.7670910730648899 and parameters: {'n_estimators': 611, 'max_depth': 3, 'min_samples_spli


--- Optuna チューニング結果 (RandomForestClassifier) ---
最適なパラメータ: {'n_estimators': 802, 'max_depth': 13, 'min_samples_split': 20, 'min_samples_leaf': 7, 'max_features': 0.5651473671734111, 'bootstrap': True}
最適なCVスコア (Accuracy): 0.8033679686288189

RandomForestClassifier モデルを全データで学習中...
--- RandomForestClassifier モデル学習完了 ---

--- RandomForestClassifier テストデータ予測完了 ---
予測された Transported の分布:
True     2144
False    2133
Name: count, dtype: int64

RandomForestClassifier の予測確率を保存しました: ../outputs/oof_rf_proba.npy

--- セル7完了 ---
次のステップ: アンサンブル (モデルの予測確率の平均化など)


In [14]:
# セル 8: ユーザー想定に基づくアンサンブル (Weighted Average based on model trust/performance)

# --- 1. 各モデルの予測確率を変数から取得 (既に計算済みと仮定) ---
# y_test_pred_proba          # LightGBM (from Cell 3)
# y_test_pred_proba_xgb      # XGBoost (from Cell 4)
# y_test_pred_proba_cat      # CatBoost (from Cell 5)
# y_test_pred_proba_hgb      # HistGradientBoosting (from Cell 6)
# y_test_pred_proba_rf       # RandomForest (from Cell 7)

# --- 2. 各モデルの重みを定義 (あなたの想定に基づく) ---
# ここでは、各モデルの Optuna チューニングで得られた CV Accuracy を基準に重みを設定
# 重みは任意に調整可能です
model_weights = {
    'LightGBM': 0.8055,
    'XGBoost': 0.8027,
    'CatBoost': 0.8133,
    'HistGradientBoosting': 0.8054,
    'RandomForest': 0.8034
}

# --- 3. 重みの正規化 (合計を1にする) ---
total_weight = sum(model_weights.values())
normalized_weights = {model: weight / total_weight for model, weight in model_weights.items()}

print("--- ユーザー想定に基づくアンサンブル (重み正規化後) ---")
for model, weight in normalized_weights.items():
    print(f"  {model}: {weight:.4f}")

# --- 4. 加重平均 (Weighted Average) ---
ensemble_proba_weighted_user = (
    normalized_weights['LightGBM'] * y_test_pred_proba +
    normalized_weights['XGBoost'] * y_test_pred_proba_xgb +
    normalized_weights['CatBoost'] * y_test_pred_proba_cat +
    normalized_weights['HistGradientBoosting'] * y_test_pred_proba_hgb +
    normalized_weights['RandomForest'] * y_test_pred_proba_rf
)

# 確率をラベルに変換 (0.5 以上を True)
y_test_pred_weighted_user = (ensemble_proba_weighted_user >= 0.5).astype(bool)

print(f"\nユーザー想定加重平均アンサンブル予測された Transported の分布:")
print(pd.Series(y_test_pred_weighted_user).value_counts())

# --- 5. 提出ファイルの作成 ---
submission_weighted_user = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_test_pred_weighted_user
})
submission_path_weighted_user = '../outputs/submissions/ver5/soya_model5.csv'
submission_weighted_user.to_csv(submission_path_weighted_user, index=False)
print(f"\nユーザー想定加重平均アンサンブル提出ファイルを保存しました: {submission_path_weighted_user}")

# --- 6. 比較のため、単純平均も再掲 (Optional) ---
print("\n--- 比較: 単純平均アンサンブル ---")
ensemble_proba_simple = (
    y_test_pred_proba +
    y_test_pred_proba_xgb +
    y_test_pred_proba_cat +
    y_test_pred_proba_hgb +
    y_test_pred_proba_rf
) / 5.0

y_test_pred_simple = (ensemble_proba_simple >= 0.5).astype(bool)
print(f"単純平均アンサンブル予測された Transported の分布:")
print(pd.Series(y_test_pred_simple).value_counts())

submission_simple = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_test_pred_simple
})
submission_path_simple = '../outputs/submissions/ver5/soya_model5.csv'
submission_simple.to_csv(submission_path_simple, index=False)
print(f"単純平均アンサンブル提出ファイルを保存しました: {submission_path_simple}")

print("\n--- セル8完了 ---")
print("次のステップ: Kaggleに提出してスコアを確認、またはさらに高度なアンサンブル (Optuna重み最適化, Stacking)")


--- ユーザー想定に基づくアンサンブル (重み正規化後) ---
  LightGBM: 0.1999
  XGBoost: 0.1992
  CatBoost: 0.2018
  HistGradientBoosting: 0.1998
  RandomForest: 0.1993

ユーザー想定加重平均アンサンブル予測された Transported の分布:
True     2208
False    2069
Name: count, dtype: int64

ユーザー想定加重平均アンサンブル提出ファイルを保存しました: ../outputs/submissions/ver5/soya_model5.csv

--- 比較: 単純平均アンサンブル ---
単純平均アンサンブル予測された Transported の分布:
True     2209
False    2068
Name: count, dtype: int64
単純平均アンサンブル提出ファイルを保存しました: ../outputs/submissions/ver5/soya_model5.csv

--- セル8完了 ---
次のステップ: Kaggleに提出してスコアを確認、またはさらに高度なアンサンブル (Optuna重み最適化, Stacking)


In [15]:
# セル 9: CatBoost単体モデルによる予測と提出

# --- 1. 必要なライブラリのインポート ---
# (ファイル操作のために os をインポート)
import os

# --- 2. モデルの確認 ---
# final_model_cat がセル5で学習済みであることを前提とします。
print("--- CatBoost 単体モデルによる予測 ---")
print("学習済み CatBoost モデル:", type(final_model_cat))

# --- 3. テストデータへの予測 (確率) ---
# セル5で既にこの処理を行っていますが、念のため再掲します。
# y_test_pred_proba_cat は既に存在するはずですが、
# もし存在しない場合は以下の行のコメントを外して実行してください。
# y_test_pred_proba_cat = final_model_cat.predict_proba(X_test_processed)[:, 1] # True の確率

# --- 4. 確率をラベルに変換 ---
# セル5で既にこの処理を行っていますが、念のため再掲します。
y_test_pred_cat_single = (y_test_pred_proba_cat >= 0.5).astype(bool)

print(f"CatBoost単体モデル予測された Transported の分布:")
print(pd.Series(y_test_pred_cat_single).value_counts())

# --- 5. 提出ファイルの作成 ---
submission_cat_single = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_test_pred_cat_single
})

# --- 修正箇所 ---
# 保存先ディレクトリを指定 (ver6)
submission_dir = '../outputs/submissions/ver6/'

# ディレクトリが存在しない場合は作成
os.makedirs(submission_dir, exist_ok=True)

# 提出ファイルのパスを指定
submission_path_cat_single = os.path.join(submission_dir, 'soya_model6.csv')
# --- 修正ここまで ---

# 提出ファイルを保存
submission_cat_single.to_csv(submission_path_cat_single, index=False)
print(f"\nCatBoost単体モデル提出ファイルを保存しました: {submission_path_cat_single}")

print("\n--- セル9完了 ---")
print("次のステップ: CatBoost単体モデルの提出ファイルを Kaggle にアップロードしてスコア確認")


--- CatBoost 単体モデルによる予測 ---
学習済み CatBoost モデル: <class 'catboost.core.CatBoostClassifier'>
CatBoost単体モデル予測された Transported の分布:
True     2197
False    2080
Name: count, dtype: int64

CatBoost単体モデル提出ファイルを保存しました: ../outputs/submissions/ver6/soya_model6.csv

--- セル9完了 ---
次のステップ: CatBoost単体モデルの提出ファイルを Kaggle にアップロードしてスコア確認
